# Lichess Data Analysis Project

To begin with, Lichess makes it's database available [here](https://database.lichess.org/). I downloaded the standard games from November 2023 with .pgn.zst extension, which I then decompressed using PeaZip.

I created a MySQL table:

In [ ]:
CREATE TABLE games_metadata(
    id INT PRIMARY KEY AUTO_INCREMENT,
    gameType VARCHAR(255),
    white VARCHAR(255),
    black VARCHAR(255),
    result VARCHAR(255),
    utcDate DATE,
    utcTime TIME,
    whiteElo INT,
    blackElo INT
)

Then I used the following python code to send the relevant metadata (name of players, their rating, the result, time control and datetime info) of each game into a MySQL table:

In [ ]:
import mysql.connector
from chess.pgn import read_game

path_to_pgn_data = "database/lichess_db_standard_rated_2023-11.pgn"
insert_into_database_sql_string = '''
        INSERT INTO games_metadata (gameType, white, black, result, utcDate, utcTime, whiteElo, blackElo) 
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
        '''
connect_params = {
    "host":"",
    "user":"",
    "password":"",
    "database":""} # This dict should contain the relevant info for the mysql connector to reach the database

def connect_to_db_and_upload_list_of_rows(list_of_tuples: list[tuple]):
    db = mysql.connector.connect(**connect_params)

    cursor = db.cursor()
    cursor.executemany(
        insert_into_database_sql_string, 
        list_of_tuples)
    db.commit()
    db.close()

# So as not to exceed memory capaity, I'm inserting chunks of 10,000 rows. If done on multiple sessions you can use the empty_iterations 
def insert_into_database(data_path: str = path_to_pgn_data, chunksize: int = 10_000, iterations: int = 100, empty_iterations: int = 0):
    with open(data_path) as file:
        for _ in range(empty_iterations):
            game = read_game(file)
        for i in range(iterations):
            list_of_rows = []
            for _ in range(chunksize):
                game = read_game(file)
                data_tuple = (game.headers["Event"].split(" ")[1], 
                            game.headers["White"],
                            game.headers["Black"], 
                            game.headers["Result"],
                            game.headers["UTCDate"].replace(".", "-"),
                            game.headers["UTCTime"],
                            game.headers["WhiteElo"],
                            game.headers["BlackElo"])

                list_of_rows.append(data_tuple)

            connect_to_db_and_upload_list_of_rows(list_of_rows)

Initially I wanted to have as big a dataset as possible, but due to certain bottlenecks I changed that to only the games in a single day (the previous code uploads games in order): November 1st.

Using the SQLTools extension in vscode and the following code I created csv's for every time control containing a column for all distinct players, a column for the aumount of white games, a column for black games, and another for the total:

In [ ]:
SELECT
    w.player AS player,
    w.num_games_as_white AS white_games,
    b.num_games_as_black AS black_games,
    w.num_games_as_white + b.num_games_as_black AS total
FROM
    (SELECT
        DISTINCT white AS player,
        COUNT(*) AS num_games_as_white
    FROM
        games_metadata
    WHERE
        utcDate = "2023-11-01" AND gameType = "Correspondence" -- The second condition can be altered to focus on a given time control or removed to look at all time controls simultaneously
    GROUP BY
        white
    ORDER BY
        num_games_as_white DESC) AS w
    INNER JOIN
    (SELECT
        DISTINCT black AS player,
        COUNT(*) AS num_games_as_black
    FROM
        games_metadata
    WHERE
        utcDate = "2023-11-01" AND gameType = "Correspondence" -- Same as the previous comment
    GROUP BY
        black
    ORDER BY
        num_games_as_black DESC) AS b
    ON w.player = b.player
ORDER BY
    total DESC,
    player

For future reference I named the csv's "MostActivePlayersAll.csv", "MostActivePlayersClassical.csv", etc. This will be important for the Pareto bar chart.

I also copied the table, first into a csv, and then into a .pickle with the correct data types, called "NovemberFirst" for easy use with pandas.

For the pie graph's data we run this code:

In [ ]:
from pandas import DataFrame, Series, read_pickle, read_csv

df: DataFrame = read_pickle("database/NovemberFirst.pickle")

total_games = len(df.index)
total_games_correspondence = len(df[df["gameType"] == "Correspondence"].index)
total_games_classical = len(df[df["gameType"] == "Classical"].index)
total_games_rapid = len(df[df["gameType"] == "Rapid"].index)
total_games_blitz = len(df[df["gameType"] == "Blitz"].index)
total_games_bullet = len(df[df["gameType"] == "Bullet"].index)
total_games_ultrabullet = len(df[df["gameType"] == "UltraBullet"].index)

print(f"There where a total of {total_games}\n")
print(f"{total_games_correspondence} correspondence games, equivalent to {round(total_games_correspondence*100/total_games, 2)}%")
print(f"{total_games_classical} classical games, equivalent to {round(total_games_classical*100/total_games, 2)}%")
print(f"{total_games_rapid} rapid games, equivalent to {round(total_games_rapid*100/total_games, 2)}%")
print(f"{total_games_blitz} blitz games, equivalent to {round(total_games_blitz*100/total_games, 2)}%")
print(f"{total_games_bullet} bullet games, equivalent to {round(total_games_bullet*100/total_games, 2)}%")
print(f"{total_games_ultrabullet} ultra bullet games, equivalent to {round(total_games_ultrabullet*100/total_games, 2)}%")

For the Pareto bar chart first we need to know how many distinct players there where across all time controls:

In [ ]:
df: DataFrame = read_pickle("database/NovemberFirst.pickle")

df = df[df["gameType"] == "Blitz"] # This line can be altered for different time controls, or deleted to get the full picture
players = []
players.extend(list(df["white"]))
players.extend(list(df["black"]))
distinct_players = list(set(players))

print(f"There where {len(distinct_players)} distinct players")

Now we can get the data fot the Pareto bar chart:

In [ ]:
df: DataFrame = read_pickle("database/NovemberFirst.pickle")

game_type = "Blitz" # This line and the next can be altered to focus on different time controls, or removed to get the full picture
df = df[df["gameType"] == game_type]
players = list(read_csv("database/MostActivePlayersBlitz.csv")["player"]) # this line should be altered like the previous one
total_players = 377134 # Here we write the number of total distinct players for the corresponding game type
total_games = 1481528 # Here we write the number of total games for the corresponding game type
for i in [5, 10, 20, 33, 50]:
    number = len(df[(df["white"].isin(players[:int(total_players*(i/100))])) | (df["black"].isin(players[:int(total_players*(i/100))]))])
    print(f"The {i}% most active {game_type.lower()} players are involved in {number} games, which is {round(number*100/total_games, 2)}% of the total")

And finally, for the distribution chart we run this:

In [ ]:
df: DataFrame = read_pickle("database/NovemberFirst.pickle")

df["avgRating"] = (df["whiteElo"] + df["blackElo"]) / 2
df["avgRatingByIncrements"] = df["avgRating"] - (df["avgRating"] % 50)
df["avgRatingByIncrements"] = df["avgRatingByIncrements"].astype(int)

n_by_avgRating: Series = df.groupby("avgRatingByIncrements")["white"].count()
print(n_by_avgRating)